In [ ]:
"""
Classifications and Regressions using features generated from the computed mask

Conduct feature selection with the corresponding classificaton model

Conduct featurer selection with the corresponding regression model

"""

In [89]:
# Project-wide Library calls
import nibabel as nib
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from skimage.util import montage 
from skimage.transform import rotate
from sklearn.model_selection import train_test_split as ttt
from sklearn.metrics import accuracy_score as AS
import torch
import torch.nn as nn 
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from tqdm import tqdm
from pylab import *
import gc
import torchmetrics 
import radiomics
from torchvision.utils import save_image
import matplotlib.image
import SimpleITK as sitk
import radiomics



In [90]:
FEAT_PATH ='log/feature_extraction/'
tf=torch.load(FEAT_PATH+'tf.pt')
ef=torch.load(FEAT_PATH+'ef.pt')
ttf=torch.load(FEAT_PATH+'ttf.pt')

ts=torch.load(FEAT_PATH+'ts.pt')
es=torch.load(FEAT_PATH+'es.pt')
tts=torch.load(FEAT_PATH+'tts.pt')

tc=torch.load(FEAT_PATH+'tc.pt')
ec=torch.load(FEAT_PATH+'ec.pt')
ttc=torch.load(FEAT_PATH+'ttc.pt')



# remove features with low variance
from sklearn.feature_selection import VarianceThreshold

sel = VarianceThreshold(threshold=5)
tf=sel.fit_transform(tf)
ef=sel.fit_transform(ef)
ttf=sel.fit_transform(ttf)

tf=torch.tensor(tf)
ef=torch.tensor(ef)
ttf=torch.tensor(ttf)

# combined all data together
F = torch.cat((tf,ef,ttf))

S = torch.cat((ts,es,tts))

C = torch.cat((tc,ec,ttc))

In [91]:

"""

Classificaton: 

Patients are divided into three classes:

short-term: survival days < 10 months
mid-term: 10 months < survival days < 15 months
long-term: survival days > 15 months

"""


'\n\nClassificaton: \n\nPatients are divided into three classes:\n\nshort-term: survival days < 10 months\nmid-term: 10 months < survival days < 15 months\nlong-term: survival days > 15 months\n\n'

In [92]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import cross_val_score







In [169]:
from sklearn.svm import SVC
# Univariate feature selection
f=SelectKBest(f_classif, k=15)

f.fit(F,C)
feat=f.transform(F)

# figure out which feature is selected 
temp=f.get_support()
print("feature selected:",temp)
clf = SVC(kernel = 'poly')

s=0
std=0
for i in range(50):
    scores=cross_val_score(clf,feat,C,scoring='accuracy',cv=5)
    s+=scores.mean()
    std+=scores.std()
print("Accuracy is: ",s/50, "with a standard deviation of ",std/50)




feature selected: [False False False  True False False False  True False False False False
  True False False False  True False False False False False False False
 False False False False False False False False False False False False
 False False  True False  True False  True False  True  True  True  True
 False  True False False  True False False False False False False  True
 False False False False  True]
Accuracy is:  0.40105105105105104 with a standard deviation of  0.010748066167071038


In [220]:
from sklearn.tree import DecisionTreeClassifier as DT
# Univariate feature selection
f=SelectKBest(f_classif, k=7)

f.fit(F,C)
feat=f.transform(F)

# figure out which feature is selected 
temp=f.get_support()
print("feature selected:",temp)

clf = DT(max_depth=30)
s=0
std=0
for i in range(50):
    scores=cross_val_score(clf,feat,C,scoring='accuracy',cv=5)
    s+=scores.mean()
    std+=scores.std()
print("Accuracy is: ",s/50, "with a standard deviation of ",std/50)


feature selected: [False False False False False False False False False False False False
 False False False False  True False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False  True False False  True
 False  True False False  True False False False False False False  True
 False False False False  True]
Accuracy is:  0.6841351351351352 with a standard deviation of  0.24257692751440288


In [178]:
# feature counts =  10
from sklearn.neighbors import KNeighborsClassifier as KNN
# Univariate feature selection
f=SelectKBest(f_classif, k=10)

f.fit(F,C)
feat=f.transform(F)

# figure out which feature is selected 
temp=f.get_support()
print("feature selected:",temp)



clf = KNN(n_neighbors=2)
s=0
std=0
for i in range(50):
    scores=cross_val_score(clf,feat,C,scoring='accuracy',cv=5)
    s+=scores.mean()
    std+=scores.std()
print("Accuracy is: ",s/50, "with a standard deviation of ",std/50)

feature selected: [False False False False False False False  True False False False False
 False False False False  True False False False False False False False
 False False False False False False False False False False False False
 False False  True False False False  True False  True False False  True
 False  True False False  True False False False False False False  True
 False False False False  True]
Accuracy is:  0.4905405405405399 with a standard deviation of  0.13792302744733745


In [224]:
from sklearn.ensemble import RandomForestClassifier
# Univariate feature selection
f=SelectKBest(f_classif, k=12)

f.fit(F,C)
feat=f.transform(F)

# figure out which feature is selected 
temp=f.get_support()
print("feature selected:",temp)


clf = RandomForestClassifier(max_depth=30)
s=0
std=0
for i in range(50):
    scores=cross_val_score(clf,feat,C,scoring='accuracy',cv=5)
    s+=scores.mean()
    std+=scores.std()
print("Accuracy is: ",s/50, "with a standard deviation of ",std/50)

feature selected: [False False False False False False False  True False False False False
 False False False False  True False False False False False False False
 False False False False False False False False False False False False
 False False  True False False False  True False  True  True  True  True
 False  True False False  True False False False False False False  True
 False False False False  True]
Accuracy is:  0.6532552552552553 with a standard deviation of  0.2591480236258735


In [188]:
# feature count = 10
from sklearn.ensemble import ExtraTreesClassifier
f=SelectKBest(f_classif, k=10)

f.fit(F,C)
feat=f.transform(F)

# figure out which feature is selected 
temp=f.get_support()
print("feature selected:",temp)


clf = ExtraTreesClassifier(n_estimators=25)
s=0
std=0
for i in range(50):
    scores=cross_val_score(clf,feat,C,scoring='accuracy',cv=5)
    s+=scores.mean()
    std+=scores.std()
print("Accuracy is: ",s/50, "with a standard deviation of ",std/50)

feature selected: [False False False False False False False  True False False False False
 False False False False  True False False False False False False False
 False False False False False False False False False False False False
 False False  True False False False  True False  True False False  True
 False  True False False  True False False False False False False  True
 False False False False  True]
Accuracy is:  0.6485165165165165 with a standard deviation of  0.2518179914128382


In [150]:
"""

Regression part

"""
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,make_scorer

from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.ensemble import ExtraTreesRegressor as ETR
from sklearn.neighbors import KNeighborsRegressor as KNR
from sklearn.feature_selection import f_regression

In [225]:
# SKlearn feature extraction and regression
# Simple Linear regression
selector = SelectFromModel(estimator=LinearRegression()).fit(F, S)
print(selector.get_support())
feat=selector.transform(F)
clf=LinearRegression()
s=0
std=0
for i in range(50):
    scores=cross_val_score(clf,feat,S,scoring=make_scorer(mean_squared_error),cv=5)
    s+=scores.mean()
    std+=scores.std()
print("MSE is: ",s/50," with a standard deviation of ",std/50)

[ True  True  True False False False False False False  True  True  True
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False  True False  True False False False False False False False False
 False False False False False]
MSE is:  119858.74687500011  with a standard deviation of  27132.604759263362


In [226]:
# KNN regressor
f=SelectKBest(f_regression, k=20)

f.fit(F,S)
feat=f.transform(F)
# figure out which feature is selected 
temp=f.get_support()
print("feature selected:",temp)
clf=RFR(n_estimators = 80)
s=0
std=0
for i in range(50):
    scores=cross_val_score(clf,feat,S,scoring=make_scorer(mean_squared_error),cv=5)
    s+=scores.mean()
    std+=scores.std()
print("MSE is: ",s/50," with a standard deviation of ",std/50)

feature selected: [False  True  True  True  True  True  True  True  True False  True  True
  True  True  True  True  True  True False False False False False False
 False False False False False False False False  True False False False
 False False False False False False False False False False False False
 False False False False False False False  True False False  True False
 False False False False  True]
MSE is:  71150.27987554405  with a standard deviation of  45444.82333604785


In [227]:
# KNN regressor
selector = SelectFromModel(estimator=RFR(n_estimators = 80)).fit(F, S)
print(selector.get_support())
feat=selector.transform(F)
clf=RFR(n_estimators = 80)
s=0
std=0
for i in range(50):
    scores=cross_val_score(clf,feat,S,scoring=make_scorer(mean_squared_error),cv=5)
    s+=scores.mean()
    std+=scores.std()
print("MSE is: ",s/50," with a standard deviation of ",std/50)

[ True False False  True False False False False False False False False
  True False  True False False False False False False False False False
 False False False False False False False False  True False False False
 False False  True False False False False False False False False False
  True False False  True False  True  True False  True False  True False
 False False False False  True]
MSE is:  72176.79246800511  with a standard deviation of  47752.050866522935


In [228]:
f=SelectKBest(f_regression, k=15)

f.fit(F,S)
feat=f.transform(F)
# figure out which feature is selected 
temp=f.get_support()
print("feature selected:",temp)
clf=ETR(n_estimators = 100)
s=0
std=0
for i in range(50):
    scores=cross_val_score(clf,feat,S,scoring=make_scorer(mean_squared_error),cv=5)
    s+=scores.mean()
    std+=scores.std()
print("MSE is: ",s/50," with a standard deviation of ",std/50)

feature selected: [False False  True  True  True  True  True  True False False False  True
  True  True  True  True  True False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False  True False False  True False
 False False False False  True]
MSE is:  69815.5810295378  with a standard deviation of  59482.99323292219


In [229]:
# Extra tree regressor
selector = SelectFromModel(estimator=ETR(n_estimators = 100)).fit(F, S)
print(selector.get_support())
feat=selector.transform(F)

clf=ETR(n_estimators = 100)
s=0
std=0
for i in range(50):
    scores=cross_val_score(clf,feat,S,scoring=make_scorer(mean_squared_error),cv=5)
    s+=scores.mean()
    std+=scores.std()
print("MSE is: ",s/50," with a standard deviation of ",std/50)

[ True  True  True  True  True  True  True False  True  True  True  True
  True  True  True False False False False False False False False False
 False False False False False False False False  True False False False
 False False False False False False False False False False False False
  True False False False False False  True  True False  True  True False
 False  True False False  True]
MSE is:  67212.90361658495  with a standard deviation of  55453.6724792246


In [241]:
from sklearn.neighbors import KNeighborsRegressor
f=SelectKBest(f_regression, k=15)

f.fit(F,S)
feat=f.transform(F)
# figure out which feature is selected 
temp=f.get_support()
print("feature selected:",temp)
clf= KNeighborsRegressor(n_neighbors=15)
s=0
std=0
for i in range(50):
    scores=cross_val_score(clf,feat,S,scoring=make_scorer(mean_squared_error),cv=5)
    s+=scores.mean()
    std+=scores.std()
print("MSE is: ",s/50," with a standard deviation of ",std/50)

feature selected: [False False  True  True  True  True  True  True False False False  True
  True  True  True  True  True False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False  True False False  True False
 False False False False  True]
MSE is:  122477.40156249989  with a standard deviation of  27375.604074228573


In [253]:
from sklearn.tree import DecisionTreeRegressor
f=SelectKBest(f_regression, k=25)

f.fit(F,S)
feat=f.transform(F)
# figure out which feature is selected 
temp=f.get_support()
print("feature selected:",temp)
clf= DecisionTreeRegressor(criterion = 'friedman_mse',max_depth=15)
s=0
std=0
for i in range(50):
    scores=cross_val_score(clf,feat,S,scoring=make_scorer(mean_squared_error),cv=5)
    s+=scores.mean()
    std+=scores.std()
print("MSE is: ",s/50," with a standard deviation of ",std/50)

feature selected: [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True False False False False False False
 False False False False False False False False  True False False False
 False False False False False False False False False False False False
 False False False False False False  True  True False False  True False
 False  True False  True  True]
MSE is:  123979.18203338207  with a standard deviation of  104072.55867255018


In [255]:
from sklearn.linear_model import SGDRegressor

f=SelectKBest(f_regression, k=15)

f.fit(F,S)
feat=f.transform(F)
# figure out which feature is selected 
temp=f.get_support()
print("feature selected:",temp)
clf= SGDRegressor()
s=0
std=0
for i in range(50):
    scores=cross_val_score(clf,feat,S,scoring=make_scorer(mean_squared_error),cv=5)
    s+=scores.mean()
    std+=scores.std()
print("MSE is: ",s/50," with a standard deviation of ",std/50)

feature selected: [False False  True  True  True  True  True  True False False False  True
  True  True  True  True  True False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False  True False False  True False
 False False False False  True]
MSE is:  1.850749126753931e+32  with a standard deviation of  1.950698889123793e+32
